This is not currents ly working in a colab NB and only
abortively in an interactive python session.

The pylan osx toggle brings up a separate window for plots and the click event
registers there.  

But it has the same problem its  has in interactive python. Once there's been a click the label shows, but it stays there and only works that first time.

In [1]:
%pylab osx
import math

import pylab
import matplotlib
import networkx as nx
import matplotlib.pyplot as plt
import urllib.request

class AnnoteFinder:
  """
  callback for matplotlib to display an annotation when points are clicked on.  The
  point which is closest to the click and within xtol and ytol is identified.
    
  Register this function like this:
    
  scatter(xdata, ydata)
  af = AnnoteFinder(xdata, ydata, annotes)
  connect('button_press_event', af)
  """

  def __init__(self, xdata, ydata, annotes, axis=None, xtol=None, ytol=None):
    self.data = zip(xdata, ydata, annotes)
    if xtol is None:
      xtol = ((max(xdata) - min(xdata))/float(len(xdata)))/2
    if ytol is None:
      ytol = ((max(ydata) - min(ydata))/float(len(ydata)))/2
    self.xtol = xtol
    self.ytol = ytol
    if axis is None:
      self.axis = pylab.gca()
    else:
      self.axis= axis
    self.drawnAnnotations = {}
    self.links = []

  def distance(self, x1, x2, y1, y2):
    """
    return the distance between two points
    """
    return math.hypot(x1 - x2, y1 - y2)

  def __call__(self, event):
    if event.inaxes:
      clickX = event.xdata
      clickY = event.ydata
      if self.axis is None or self.axis==event.inaxes:
        annotes = []
        for x,y,a in self.data:
          if  clickX-self.xtol < x < clickX+self.xtol and  clickY-self.ytol < y < clickY+self.ytol :
            annotes.append((self.distance(x,clickX,y,clickY),x,y, a) )
        if annotes:
          annotes.sort()
          distance, x, y, annote = annotes[0]
          self.drawAnnote(event.inaxes, x, y, annote)
          for l in self.links:
            l.drawSpecificAnnote(annote)

  def drawAnnote(self, axis, x, y, annote):
    """
    Draw the annotation on the plot
    """
    if (x,y) in self.drawnAnnotations:
      markers = self.drawnAnnotations[(x,y)]
      for m in markers:
        m.set_visible(not m.get_visible())
      self.axis.figure.canvas.draw()
    else:
      #t = axis.text(x,y, "(%3.2f, %3.2f) - %s"%(x,y,annote), )
      t = axis.text(x,y, "%s"%(annote,), bbox = dict(boxstyle = 'round,pad=0.15', fc = 'salmon', alpha = .75))
      m = axis.scatter([x],[y], marker='d', c='r', zorder=100)
      self.drawnAnnotations[(x,y)] =(t,m)
      self.axis.figure.canvas.draw()

  def drawSpecificAnnote(self, annote):
    annotesToDraw = [(x,y,a) for x,y,a in self.data if a==annote]
    for x,y,a in annotesToDraw:
      self.drawAnnote(self.axis, x, y, a)


def url_fetch_networkx_graph (url):
    with urllib.request.urlopen(url) as filehandle:
         G = nx.read_gml(filehandle)
    return G


############################################################
############################################################
##
##
##
##
##
############################################################
############################################################
      

def linkAnnotationFinders(afs):
  for i in range(len(afs)):
    allButSelfAfs = afs[:i]+afs[i+1:]
    afs[i].links.extend(allButSelfAfs)

print('Defined')

Populating the interactive namespace from numpy and matplotlib
Defined


# The code

Dies not work in a notebook. 

Works rather indequately elsewhere.  One node shows itsam
labnel when clicked on.  The label stays there.  After
that all other nodes are inert.

In [2]:
import os.path

github_networks_data = 'https://raw.githubusercontent.com/gawron/python-for-social-science/master/networks/'
ak_url = os.path.join(github_networks_data, 'anna.gml')
ak = url_fetch_networkx_graph(ak_url)

ak_pos = nx.spring_layout(ak,scale=1.0,iterations=500)
ak_il = ak_pos.items()

labels = [label for (label,coords) in ak_il]
X = [x for (label,(x,y)) in ak_il]
Y = [y for (label,(x,y)) in ak_il]


#nx.draw_networkx_edges(ak,ak_pos)
plt.figure(figsize=(8,8))
nx.draw_networkx(ak,ak_pos, with_labels=False,node_size=50,node_color="red") 

af =  AnnoteFinder(X,Y, labels)
pylab.connect('button_press_event', af)



8

In [ ]:

lm_pos = nx.spring_layout(lm,scale=1.0)
lm_il = list(lm_pos.items())

labels = [label for (label,coords) in lm_il]
X = [x for (label,(x,y)) in lm_il]
Y = [y for (label,(x,y)) in lm_il]
     

# An alternative is to draw just the edges.  The nodes are still clickable, but harder to find.
#nx.draw_networkx_edges(ego_network,ego_pos)
nx.draw_networkx(lm,lm_pos, with_labels=False,node_size=20,node_color="lightgray") 
af = AnnoteFinder(X,Y, labels)
pylab.connect('button_press_event', af)

In [3]:
fig, ax = plt.subplots()
ax.plot(np.random.rand(10))

def onclick(event):
    print('%s click: button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          ('double' if event.dblclick else 'single', event.button,
           event.x, event.y, event.xdata, event.ydata))

cid = fig.canvas.mpl_connect('button_press_event', onclick)

single click: button=1, x=554, y=503, xdata=3.482056, ydata=0.561002
single click: button=1, x=406, y=315, xdata=2.005040, ydata=0.322263
single click: button=1, x=275, y=289, xdata=0.697681, ydata=0.289246
single click: button=1, x=305, y=147, xdata=0.997077, ydata=0.108921
